In [1]:
from symmer.chem.chem_utils import xyz_from_pubchem, Draw_molecule
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

/home/tim/anaconda3/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
2022-05-25 17:33:29.492827: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 17:33:29.528462: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-25 17:33:29.531533: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [40]:
geometry=[
    ('H', (0.0,0.0,0.0)),
    ('H', (2.45366053071732,0.0,0.0)),
    ('H', (2.45366053071732,2.45366053071732,0.0)),
    ('H', (0.0,2.45366053071732,0.0))
     ] 

geometry=[('N', (0,0,0))]#, ('N', (1000,0,0))]

def list_to_xyz(geometry) -> str:
    """ Convert the geometry stored as a list to xyz string
    """
    xyz_file = str(len(geometry))+'\n '

    for atom, coords in geometry:
        xyz_file += '\n'+atom+'\t'
        xyz_file += '\t'.join(list(map(str, coords)))
    
    return xyz_file

xyz_file = list_to_xyz(geometry) #'4\n \nH\t0\t0\t0\nH\t2.45366053071732\t0\t0\nH\t2.45366053071732\t2.45366053071732\t0\nH\t0\t2.45366053071732\t0'

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [41]:
print(xyz_file)

1
 
N	0	0	0


In [42]:
from symmer.chem import PySCFDriver

In [43]:
basis = 'sto-3g' # 'STO-3G'#
convergence = 1e-6
charge= 0
max_hf_cycles=50
ram = 8_000
run_mp2  = True
run_cisd = False
run_ccsd = True
run_fci  = True

In [50]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=0,
                       spin=1,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [51]:
pyscf_obj.run_pyscf()

In [52]:
pyscf_obj.pyscf_hf.converged

True

In [53]:
pyscf_obj.pyscf_ccsd.converged

True

In [54]:
# T1 = pyscf_obj.pyscf_ccsd.t1
# T2 = pyscf_obj.pyscf_ccsd.t2

# from pyscf import cc
# ccsd = cc.CCSD(pyscf_obj.pyscf_hf)
# ccsd.kernel(t1= T1, t2=T2)
# ccsd.converged


In [55]:
from symmer.chem import FermionicHamilt

H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

In [56]:
H_ferm.build_fermionic_hamiltonian_operator()
H_f = H_ferm.fermionic_molecular_hamiltonian

In [57]:
H_ferm.fock_spin_mo_basis.diagonal()

array([-15.3220493 , -15.29989026,  -0.93701256,  -0.78753493,
        -0.35301588,  -0.31189736,  -0.43525292,   0.32755162,
         0.20419606,   0.24531458])

In [58]:
pyscf_obj.pyscf_mp2.e_corr

-0.011925312762693503

In [59]:
from symmer.chem.fermionic_ham import get_T2_mp2

In [60]:
T2_ferm = get_T2_mp2(pyscf_obj.pyscf_mp2.t2)
mp2_ket = get_sparse_operator(T2_ferm) @ H_ferm.hf_ket

In [61]:
v_pert = H_ferm.get_perturbation_correlation_potential()

In [62]:
H_ferm.hf_ket.conj().T @ mp2_ket

0j

In [63]:
H_ferm.hf_ket.conj().T @ v_pert @ mp2_ket - pyscf_obj.pyscf_mp2.e_corr

0j

In [64]:
H_ferm.hf_ket.conj().T @ v_pert @ mp2_ket - pyscf_obj.pyscf_mp2.e_corr

0j

In [65]:
from symmer.symplectic import QuantumState, PauliwordOp

hf_state = H_ferm.hf_comp_basis_state
HF_state = QuantumState([hf_state], [1])

print(HF_state)

 1.0000000000 |1111111000>


In [66]:
from openfermion import jordan_wigner
from symmer.utils import QubitOperator_to_dict

T2 = PauliwordOp(QubitOperator_to_dict(jordan_wigner(T2_ferm), H_ferm.n_qubits))

In [67]:
print(T2 * HF_state)

-0.0007131640+0.0000000000j |0011111011> +
-0.0018671833+0.0000000000j |0110111011> +
 0.0018529366+0.0000000000j |1001111011> +
-0.0687551188+0.0000000000j |1100111011> +
-0.0368966592+0.0000000000j |1111001011>


In [68]:
from symmer.projection import QubitTapering

H_q = PauliwordOp(QubitOperator_to_dict(jordan_wigner(H_f), H_ferm.n_qubits))

taper_hamiltonian = QubitTapering(H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated via a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)
print('\nonto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
hf_array = H_ferm.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
#sor_tap = taper_hamiltonian.taper_it(aux_operator=molecule.SOR_q, ref_state=hf_array)
#sor_tap.coeff_vec/=np.linalg.norm(sor_tap.coeff_vec)
#ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

 1 ZIZIIZIZIZ 
 1 IZIZIZIZIZ 
 1 IIIIZZIIII 
 1 IIIIIIZZII 
 1 IIIIIIIIZZ

and may be rotated via a sequence of Clifford pi/2 rotations

None

onto the single-qubit Pauli operators

-1 IIIIIIIIXI 
-1 IIIIIIXIII 
-1 IIIIXIIIII 
-1 IXIIIIIIII 
-1 XIIIIIIIII

The symetry sector corresponding with the reference [1 1 1 1 1 1 1 0 0 0] is [-1 -1  1 -1  1]
The tapered Hartree-Fock state is  1.0000000000 |11100>


In [69]:
from symmer.utils import exact_gs_energy

gs_energy, gs_vector = exact_gs_energy(H_q.to_sparse_matrix)

In [70]:
from itertools import product

for comb in product([+1,-1], repeat=n_taper):
    ham_tap = taper_hamiltonian.taper_it(sector=comb)
    tap_gs  = exact_gs_energy(ham_tap.to_sparse_matrix)[0]
    error = tap_gs - pyscf_obj.pyscf_fci.e_tot 
    print(f'The energy in sector {comb} is {tap_gs}, with error {error} w.r.t. FCI')

The energy in sector (1, 1, 1, 1, 1) is -53.268102989094295, with error 0.45090717349964393 w.r.t. FCI
The energy in sector (1, 1, 1, 1, -1) is -53.59565460717697, with error 0.12335555541697119 w.r.t. FCI
The energy in sector (1, 1, 1, -1, 1) is -53.595654607176904, with error 0.12335555541703513 w.r.t. FCI
The energy in sector (1, 1, 1, -1, -1) is -53.350340026038936, with error 0.3686701365550036 w.r.t. FCI
The energy in sector (1, 1, -1, 1, 1) is -53.595654607176975, with error 0.12335555541696408 w.r.t. FCI
The energy in sector (1, 1, -1, 1, -1) is -53.350340026038964, with error 0.36867013655497516 w.r.t. FCI
The energy in sector (1, 1, -1, -1, 1) is -53.350340026038936, with error 0.3686701365550036 w.r.t. FCI
The energy in sector (1, 1, -1, -1, -1) is -53.71901016259391, with error 2.842170943040401e-14 w.r.t. FCI
The energy in sector (1, -1, 1, 1, 1) is -53.02760904295053, with error 0.6914011196434089 w.r.t. FCI
The energy in sector (1, -1, 1, 1, -1) is -52.80813614169475, wi

In [71]:
tap_gs = exact_gs_energy(ham_tap.to_sparse_matrix)[0]
print(tap_gs, pyscf_obj.pyscf_fci.e_tot)

-53.719010162593904 -53.71901016259394


In [72]:
tap_gs - pyscf_obj.pyscf_fci.e_tot

3.552713678800501e-14

In [73]:
pyscf_obj.pyscf_fci.nelec

(4, 3)

In [74]:
print(pyscf_obj.pyscf_hf.e_tot - pyscf_obj.pyscf_fci.e_tot)
print(pyscf_obj.pyscf_mp2.e_tot - pyscf_obj.pyscf_fci.e_tot)
print(pyscf_obj.pyscf_ccsd.e_tot - pyscf_obj.pyscf_fci.e_tot)

0.1644740738892594
0.15254876112656746
0.12335555570629708


In [75]:
taper_hamiltonian.stabilizers.update_sector(np.array([1, 1, 0, 1, 1, 0, 0, 0]))
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
ham_tap = taper_hamiltonian.taper_it(ref_state=np.array([1, 1, 0, 1, 1, 0, 0, 0]))
exact_gs_energy(ham_tap.to_sparse_matrix)[0] - pyscf_obj.pyscf_fci.e_tot

ValueError: operands could not be broadcast together with shapes (5,10) (8,) 

In [76]:
T2_state = T2*HF_state

for term in T2_state.state_matrix:
    taper_hamiltonian.stabilizers.update_sector(term)
    print(f'The symetry sector corresponding with the reference {term} is {taper_hamiltonian.stabilizers.coeff_vec}')

The symetry sector corresponding with the reference [0 0 1 1 1 1 1 0 1 1] is [-1 -1  1 -1  1]
The symetry sector corresponding with the reference [0 1 1 0 1 1 1 0 1 1] is [-1 -1  1 -1  1]
The symetry sector corresponding with the reference [1 0 0 1 1 1 1 0 1 1] is [-1 -1  1 -1  1]
The symetry sector corresponding with the reference [1 1 0 0 1 1 1 0 1 1] is [-1 -1  1 -1  1]
The symetry sector corresponding with the reference [1 1 1 1 0 0 1 0 1 1] is [-1 -1  1 -1  1]


In [77]:
from symmer.utils import exact_gs_energy
from symmer.symplectic import array_to_QuantumState

gs = array_to_QuantumState(exact_gs_energy(H_q.to_sparse_matrix)[1]).cleanup(zero_threshold=1e-8)
print(gs)

-0.0237610795-0.2671649690j |1111010101> +
 0.1036707163-0.0714405514j |1111010110> +
 0.1036707163-0.0714405514j |1111011001> +
-0.4408250397+0.0497532387j |1111011010> +
 0.1036707163-0.0714405514j |1111100101> +
-0.4408250397+0.0497532387j |1111100110> +
-0.4408250397+0.0497532387j |1111101001> +
 0.1835805347+0.5063556727j |1111101010>


In [78]:
gs.sectors_present(taper_hamiltonian.symmetry_generators)

array([-0.32469602+0.j, -0.32469602+0.j, -1.        +0.j, -1.        +0.j,
       -1.        +0.j])

In [39]:
op = PauliwordOp(['XZX'],[1])
r_state = QuantumState(np.random.randint(0,2,(10,3)), np.random.rand(10)+1j*np.random.rand(10))
for op in taper_hamiltonian.symmetry_generators:
    op.coeff_vec[0]=1
    print(op)
    print()
    print(HF_state.conjugate * op * HF_state)
    print((HF_state.state_op.to_sparse_matrix.H @ op.to_sparse_matrix @ HF_state.state_op.to_sparse_matrix)[-1,-1])

1.000+0.000j ZIIZIZZI

(1+0j)
(1+0j)
1.000+0.000j IZIZIZIZ

(1+0j)
(1+0j)
1.000+0.000j IIZZIIII

(1+0j)
(1+0j)
1.000+0.000j IIIIZZII

(1+0j)
(1+0j)


In [314]:
print(HF_state)

 1.0000000000 |1111111000>


In [303]:
(r_state.state_op.to_sparse_matrix.H @ op.to_sparse_matrix @ r_state.state_op.to_sparse_matrix)[-1,-1]

(-3.9462117730293147-5.551115123125783e-17j)

In [137]:
np.random.randint(0,2,(1,20))

array([[1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]])

In [82]:
from scipy.sparse.linalg import eigsh

eigvecs, eigvals = eigsh(H_q.to_sparse_matrix)

In [84]:
eigvecs,eigvals = eigvals,eigvecs

In [99]:
eigvals[order]

array([-53.71901016, -53.71901016, -53.71901016, -53.71901016,
       -53.59565461, -53.59565461])

In [102]:
taper_hamiltonian.stabilizers.update_sector([1,1,1,1,0,1,0,1,0,1])
print(f'\nThe symetry sector corresponding with the reference {[1,1,1,1,0,1,0,1,0,1]} is {taper_hamiltonian.stabilizers.coeff_vec}')


The symetry sector corresponding with the reference [1, 1, 1, 1, 0, 1, 0, 1, 0, 1] is [-1 -1 -1 -1 -1]


In [114]:
for row in array_to_QuantumState(eigvecs.T[order][0]).cleanup(zero_threshold=1e-8).state_matrix:
    taper_hamiltonian.stabilizers.update_sector(row)
    print(f'The symetry sector corresponding with the reference {row} is {taper_hamiltonian.stabilizers.coeff_vec}')

The symetry sector corresponding with the reference [1 1 1 1 0 1 0 1 0 1] is [-1 -1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 0 1 0 1 1 0] is [ 1  1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 0 1 1 0 0 1] is [ 1  1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 0 1 1 0 1 0] is [-1 -1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 1 0 0 1 0 1] is [ 1  1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 1 0 0 1 1 0] is [-1 -1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 1 0 1 0 0 1] is [-1 -1 -1 -1 -1]
The symetry sector corresponding with the reference [1 1 1 1 1 0 1 0 1 0] is [ 1  1 -1 -1 -1]


In [100]:
order = np.argsort(eigvals)
for vec in eigvecs.T[order]:
    print(array_to_QuantumState(vec).cleanup(zero_threshold=1e-8))
    print()

 0.3164680304+0.6023396146j |1111010101> +
 0.0236638629+0.1152160462j |1111010110> +
 0.0236638629+0.1152160462j |1111011001> +
-0.2217323129-0.1101427450j |1111011010> +
 0.0236638629+0.1152160462j |1111100101> +
-0.2217323129-0.1101427450j |1111100110> +
-0.2217323129-0.1101427450j |1111101001> +
-0.5500815714+0.0951376027j |1111101010>

-0.2014684997+0.4433599935j |1111010101> +
-0.1047827929-0.3155325682j |1111010110> +
-0.1047827929-0.3155325682j |1111011001> +
-0.1451206646-0.0570686196j |1111011010> +
-0.1047827929-0.3155325682j |1111100101> +
-0.1451206646-0.0570686196j |1111100110> +
-0.1451206646-0.0570686196j |1111101001> +
 0.4391899422+0.4066735112j |1111101010>

 0.1482897596-0.3906071488j |1111010101> +
-0.1321226273-0.3651915727j |1111010110> +
-0.1321226273-0.3651915727j |1111011001> +
-0.2108096434-0.0808182838j |1111011010> +
-0.1321226273-0.3651915727j |1111100101> +
-0.2108096434-0.0808182838j |1111100110> +
-0.2108096434-0.0808182838j |1111101001> +
-0.3418647064

In [69]:
from symmer.chem.fermionic_ham import FermioniCC

CC_ferm = FermioniCC(pyscf_obj.pyscf_ccsd)
CC_ferm.build_operator()
T_f = CC_ferm.fermionic_cc_operator
T_q = PauliwordOp(QubitOperator_to_dict(jordan_wigner(T_f), H_ferm.n_qubits))

In [70]:
# from scipy.sparse.linalg import expm

# T_exp = expm(T_q.to_sparse_matrix)

In [71]:
print(T_q*HF_state)

-1.0000000227+0.0000000000j |1111001011>


In [72]:
CC_state_1 = T_q*HF_state

for term in CC_state_1.state_matrix:
    taper_hamiltonian.stabilizers.update_sector(term)
    print(f'The symetry sector corresponding with the reference {term} is {taper_hamiltonian.stabilizers.coeff_vec}')

The symetry sector corresponding with the reference [1 1 1 1 0 0 1 0 1 1] is [-1 -1  1 -1  1]


In [74]:
from scipy.sparse.linalg import expm

T_exp = expm(T_q.to_sparse_matrix)

cc_ket = T_exp @ H_ferm.hf_ket
CC_state_full = array_to_QuantumState(cc_ket).cleanup(zero_threshold=1e-8)
print(CC_state_full)
for term in CC_state_full.state_matrix:
    taper_hamiltonian.stabilizers.update_sector(term)
    print(f'The symetry sector corresponding with the reference {term} is {taper_hamiltonian.stabilizers.coeff_vec}')

/home/tim/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/tim/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:216: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


-1.0000000227+0.0000000000j |1111001011> +
 1.0000000000+0.0000000000j |1111111000>
The symetry sector corresponding with the reference [1 1 1 1 0 0 1 0 1 1] is [-1 -1  1 -1  1]
The symetry sector corresponding with the reference [1 1 1 1 1 1 1 0 0 0] is [-1 -1  1 -1  1]


In [77]:
(H_ferm.hf_ket.conj().T @ H_q.to_sparse_matrix @ cc_ket) - pyscf_obj.pyscf_ccsd.e_tot

(-1.2227872048242716e-09+0j)

In [76]:
pyscf_obj.pyscf_ccsd.e_tot

-53.59565460688764

In [15]:
hf_ket = H_ferm.hf_ket
H_mat = H_ferm.get_sparse_ham()

In [16]:
from pyscf.cc.addons import spatial2spin

# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [17]:
# singles

t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T
ccsd_single_amps.shape

(8, 8)

In [18]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[4, 0, 5, 1], -0.5456412208879349],
 [[4, 0, 6, 2], 2.208863578892679e-08],
 [[4, 0, 7, 3], -1.7451411549163962e-08],
 [[4, 1, 5, 0], 0.5456412208879349],
 [[4, 1, 7, 2], 3.954004733809075e-08],
 [[4, 2, 5, 3], 0.0366321093272368],
 [[4, 2, 6, 0], -2.208863578892679e-08],
 [[4, 2, 7, 1], -3.954004733809075e-08],
 [[4, 3, 5, 2], -0.0366321093272368],
 [[4, 3, 7, 0], 1.7451411549163962e-08],
 [[5, 0, 4, 1], 0.5456412208879349],
 [[5, 0, 6, 3], 3.954004733809075e-08],
 [[5, 1, 4, 0], -0.5456412208879349],
 [[5, 1, 6, 2], -1.7451411549163962e-08],
 [[5, 1, 7, 3], 2.208863578892679e-08],
 [[5, 2, 4, 3], -0.0366321093272368],
 [[5, 2, 6, 1], 1.7451411549163962e-08],
 [[5, 3, 4, 2], 0.0366321093272368],
 [[5, 3, 6, 0], -3.954004733809075e-08],
 [[5, 3, 7, 1], -2.208863578892679e-08],
 [[6, 0, 4, 2], -2.208863578892679e-08],
 [[6, 0, 5, 3], -3.954004733809075e-08],
 [[6, 0, 7, 1], 0.0006616412681631195],
 [[6, 1, 5, 2], 1.7451411549163962e-08],
 [[6, 1, 7, 0], -0.0006616412681631195],
 [[6, 

In [19]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[4, 0], -3.912181678103236e-15],
 [[4, 2], -6.110293692729721e-08],
 [[5, 1], -3.912181678103236e-15],
 [[5, 3], -6.110293692729721e-08],
 [[6, 0], -0.4435278092467373],
 [[6, 2], 4.3248127744901715e-15],
 [[7, 1], -0.4435278092467373],
 [[7, 3], 4.3248127744901715e-15]]

In [20]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [21]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [22]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))

In [96]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

(-1.8261563137028727+0j)

In [97]:
pyscf_obj.pyscf_ccsd.e_tot 

-1.8261563137028722

In [98]:
# intermediate normalization
hf_ket.conj().T  @ ccsd_ket


(1+0j)

In [99]:
normalized_state = ccsd_ket / np.sqrt(ccsd_ket.conj().T@ccsd_ket)

E_from_vec = normalized_state.conj().T @ H_mat @ normalized_state
E_from_vec

(-1.7418931709154082+0j)

In [100]:
abs(pyscf_obj.pyscf_ccsd.e_tot - E_from_vec)

0.08426314278746405

In [86]:
for ind, amp in enumerate(normalized_state):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(0.5276099995527699+0j)|00001111>
(-1.8433902498537798e-08+0j)|00011011>
(1.8433902498537798e-08+0j)|00100111>
(0.08576206597664797+0j)|00110011>
(-0.4725840345446496+0j)|00111100>
(0.014071962063256225+0j)|01001110>
(2.250980991942932e-08+0j)|01100110>
(0.19207141387290994+0j)|01110010>
(-0.014071962063256225+0j)|10001101>
(2.2509809919429316e-08+0j)|10011001>
(-0.19207141387290996+0j)|10110001>
(-0.47771932255267385+0j)|11000011>
(0.03172735005536297+0j)|11001100>
(2.6460860497891207e-08+0j)|11011000>
(-2.6460860497891207e-08+0j)|11100100>
(0.43305382406373405+0j)|11110000>


In [87]:
trotter_op = np.eye(2**H_ferm.n_qubits)
for op in list(generator_t2):
    trotter_op*= expm(get_sparse_operator(op, n_qubits=H_ferm.n_qubits))

    
ccsd_trotter = trotter_op @ hf_ket


normalized_trotter = ccsd_trotter / np.sqrt(ccsd_trotter.conj().T@ccsd_trotter)

normalized_trotter.conj().T @ H_mat @ normalized_trotter

(-1.5162206379375986+0j)

In [88]:
for ind, amp in enumerate(normalized_trotter):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(0.8277926697224702+0j)|00001111>
(0.0004549263477769256+0j)|00110011>
(-0.3751679040580851+0j)|00111100>
(-1.5187538762098153e-08+0j)|01011010>
(2.7186649612082093e-08+0j)|01100110>
(-1.199911084998394e-08+0j)|01101001>
(-1.199911084998394e-08+0j)|10010110>
(2.7186649612082093e-08+0j)|10011001>
(-1.5187538762098153e-08+0j)|10100101>
(-0.3792446292495347+0j)|11000011>
(0.025187231373688078+0j)|11001100>
(0.1718931276157827+0j)|11110000>


In [89]:
ccsd_ansatz.shape

(256, 256)

In [90]:
zero_state = np.zeros((H_mat.shape[0],1))
zero_state[0]=1

# T = generator_t1 + 0.5*generator_t2
hf_ket.conj().T @ np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat @ ccsd_ansatz @ hf_ket

matrix([[-1.82615631+0.j]])

In [33]:
vals, vecs = np.linalg.eigh(H_mat.todense())
min(vals)

-1.8743108952541805

In [34]:
H_rot =  np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat.todense() @ ccsd_ansatz 
# H_rot =  H_mat.todense() @ ccsd_ansatz 
# H_rot = np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat.todense() 

vals, vecs = np.linalg.eigh(H_rot)
min(vals)

-2.096006827134742

In [40]:
ccsd_ansatz_inv = expm(get_sparse_operator(-1*T, n_qubits=H_ferm.n_qubits))
H_rot =  ccsd_ansatz_inv @ H_mat @ ccsd_ansatz 

vals, vecs = np.linalg.eigh(H_rot.todense())
print(min(vals))

-2.0960068271347416


In [41]:
hf_ket.conj().T @ H_rot @ hf_ket

(-1.826156313702873+0j)

In [77]:
from symred.symplectic_form import PauliwordOp
conv_dic = {0: 'I', 1:'X'}
P = ''.join([conv_dic[i] for i in H_ferm.hf_comp_basis_state])
HF_ansatz = PauliwordOp([P],[1])
print(HF_ansatz)

HF_ans_mat = HF_ansatz.to_sparse_matrix

1.0000000000+0.0000000000j XXXXIIII


In [66]:
np.allclose(HF_ans_mat.conj().T.todense(),  HF_ans_mat.todense())

True

In [75]:
np.allclose(HF_ans_mat@zero_state, hf_ket.reshape(-1,1))

True

In [76]:
# H_rot =  HF_ans_mat.conj().T @ ccsd_ansatz_inv @ H_mat @ ccsd_ansatz @ HF_ans_mat
# vals, vecs = np.linalg.eigh(H_rot.todense())
# print(min(vals))


fin_state = ccsd_ansatz @ HF_ans_mat@zero_state
fin_state.conj().T @ H_mat @ fin_state 


array([[-9.28833265+0.j]])

In [82]:
zero_state = np.zeros((H_mat.shape[0],1))
zero_state[0]=1

zero_state.conj().T @HF_ans_mat.conj().T @ np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat @ ccsd_ansatz @ HF_ans_mat@ zero_state

matrix([[-1.82615631+0.j]])

In [83]:
zero_state.conj().T @HF_ans_mat.conj().T @ ccsd_ansatz.todense().conj().T @ H_mat @ ccsd_ansatz @ HF_ans_mat@ zero_state



matrix([[-9.28833265+0.j]])

In [130]:
state1 = ccsd_ansatz @ HF_ans_mat@ zero_state

state2 = state1.conj().T

state3 = zero_state.conj().T @HF_ans_mat.conj().T @ np.linalg.pinv(ccsd_ansatz.todense())

print(np.allclose(state2,
           state3))

state3 @ state1

False


matrix([[1.+0.j]])

In [ ]:
from openfermion import hermitian_conjugated
T = generator_t1 + 0.5*generator_t2

T_dagger = hermitian_conjugated(T)

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))